In [22]:
import torch
from torch.nn import ZeroPad2d
from icecream import ic

In [1]:
import torch
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

torch.set_default_dtype(torch.float64)
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
device = torch.device('cpu')
dtype = torch.float64

%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from seqm.seqm_functions.fock import fock
from seqm.seqm_functions.pack import unpack
import seqm.seqm_functions.pack as pack
import torch.nn.functional as F

Decorating your function! <function KSA_XL_BOMD.one_step at 0x7f98d3f6d6c0>


# log

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# PASCAL 1 COULD BE INCORRECT

In [101]:
#CH4

# 6       -0.507912712      0.000000000     -2.289086059
# 1        0.022521233     -0.951333822     -2.212896028
# 1        0.208634973      0.821859802     -2.231579410
# 1       -1.224460397      0.083844265     -1.469494278
# 1       -1.038346658      0.045629756     -3.242374518


# H2O
                            #   [
                            #    [-0.368758544,      0.000000000,     -1.211413881],
                            #    [-0.368758544,      0.759337000,     -0.615370881],
                            #    [-0.368758544,     -0.759337000,     -0.615370881],
                            #   ]

In [138]:
mol.C_mo

tensor([[[    -0.74608,     -0.00000,      0.00000,      0.00000,      0.66585,     -0.00000,     -0.00000,     -0.00000],
         [     0.00000,     -0.54109,      0.08919,      0.44609,      0.00000,      0.50237,      0.08732,      0.49017],
         [     0.00000,      0.25267,      0.63536,      0.17945,      0.00000,      0.23706,      0.57039,     -0.34457],
         [    -0.00000,     -0.37830,      0.29681,     -0.51819,      0.00000,     -0.43783,      0.40903,      0.37586],
         [    -0.33293,     -0.44140,     -0.42418,      0.02097,     -0.37304,     -0.00602,      0.36873,     -0.48867],
         [    -0.33293,     -0.16015,      0.47865,      0.34706,     -0.37304,     -0.41985,     -0.43986,     -0.07108],
         [    -0.33293,      0.07844,      0.18461,     -0.57877,     -0.37304,      0.55406,     -0.25406,      0.05713],
         [    -0.33293,      0.52310,     -0.23907,      0.21074,     -0.37304,     -0.12820,      0.32518,      0.50261]]])

In [124]:
%%time

### create molecule object:

#CH4
species = torch.as_tensor([[6,1,1,1,1]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [-0.507912712,      0.000000000,     -2.289086059],
                               [0.022521233,     -0.951333822,     -2.212896028],
                               [0.208634973,      0.821859802,     -2.231579410],
                               [-1.224460397,      0.083844265,     -1.469494278],
                               [-1.038346658,      0.045629756,     -3.242374518],
                              ]
                            ], device=device)

# H2O 

# species = torch.as_tensor([[8,1,1]], # zero-padding for batching
#                           dtype=torch.int64, device=device)

# coordinates = torch.tensor([
#                               [
#                                [-0.368758544,      0.000000000,     -1.211413881],
#                                [-0.368758544,      0.759337000,     -0.615370881],
#                                [-0.368758544,     -0.759337000,     -0.615370881],
#                               ]
#                             ], device=device)

#H2
# species = torch.as_tensor([[1,1]], # zero-padding for batching
#                           dtype=torch.int64, device=device)

# coordinates = torch.tensor([
#                               [
#                                [-0.368758544,      0.759337000,     -0.615370881],
#                                [-0.368758544,     -0.759337000,     -0.615370881],
#                               ]
#                             ], device=device)


const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'PM3',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                   'excited' : True,
                   }

mol = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on m:
esdriver(mol)

species tensor([[6, 1, 1, 1, 1]])
species.shape torch.Size([1, 5])
species tensor([[6, 1, 1, 1, 1]])
species.shape torch.Size([1, 5])
tore Parameter containing:
tensor([0., 1., 0., 1., 2., 3., 4., 5., 6., 7., 0., 1., 2., 3., 4., 5., 6., 7., 0.])
qn Parameter containing:
tensor([0., 1., 0., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3., 3., 3., 3., 3., 0.])
nmol 1
molsize 5
P0 torch.Size([1, 20, 20])
tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [386]:
mol.dm.shape

torch.Size([1, 20, 20])

In [143]:
def gen_V(N_cis, N_rpa, n_V_start):
    
    
  #  N_cis = 6
    # returns vexp1
    print('mol Z', mol.Z)
    rrwork = torch.zeros(N_rpa * 4, device=device)
    i = 0
    for ip in range(mol.nocc):
        for ih in range(mol.nvirt):
            rrwork[i] = mol.e_mo[0][mol.nocc + ih] - mol.e_mo[0][ip]  # !Lancos vectors(i) ???
            i += 1                                                                      #  TODO: [0] should be replaced by m batch index
                                                                                        # ? why 64 if most are 0?

    rrwork_sorted, indices = torch.sort(rrwork[:N_cis], descending=False)
    print("rrwork_sorted", rrwork_sorted)
    print("indices shape", indices.shape)
    print("indices", indices)

    vexp1 = torch.zeros((N_cis, N_cis), device=device)
    
    # print("vexp1.shape", vexp1.shape)
    # print("vexp1", vexp1)
    row_idx = torch.arange(0, int(N_cis), device=device)
    col_idx = indices[:N_cis]

    print("row_idx", row_idx)
    print("col_idx", col_idx)

    vexp1[row_idx, col_idx] = 1.0 
    print("vexp1.shape", vexp1.shape)
    print("vexp1", vexp1)
    # vexp = torch.zeros((N_rpa, nd), device=device)

    return vexp1

# TODO: check whether L_xi should be regenerated during expansion each time or just part of it

In [4]:
torch.set_printoptions(precision=5, linewidth=200, sci_mode=False)

In [5]:
form_L_xi?

Object `form_L_xi` not found.


In [187]:
def davidson(n_V_max, n_iter, tol):
    
    N_exc = 3
    n_V_start = k_dim = N_exc * 2 # dimension of Krylov subspace, analogue of nd1  
    k_dim = n_V_start # initialize
    N_cis = mol.nocc * mol.nvirt
    N_rpa = 2 * mol.nocc * mol.nvirt
    keep_n = 3
    
    term = False  # terminate algorithm
    iter = 0

    L_xi = torch.zeros((N_rpa, n_V_start), device=device)

    V = gen_V(N_cis, N_rpa, n_V_start) # generate initial guess, V here #! should be renamed
    print('V shape', V.shape)
    # V =  torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
    #             [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
    #             [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    #             [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    
    #V = V[:k_dim] # take only part
    print('V shape', V.shape)
    print('V VEXP1\n', V)
    while iter < n_iter and not term: # big loop
        
        # print('=================================')
        print(' *********** ITERATION : ', iter)
        # print('SUBSPACE SIZE V: ', V.shape)
        # print('=================================', flush=True)
        iter += 1
#        V = torch.linalg.qr(V)[0]    # most likely, do not need. create_subspace_matrix already orthogonalizes; 
                                      # later on done before adding to matrix
        ### project original matrix in a subspace matrix
        
        for i in range(k_dim):  # i=nd1_old+1,nd1  
            # print(f'==============Lxi {i}===================')
            L_xi[:,i] = form_L_xi(V[:,i], mol, N_cis, N_rpa)
            # print('L_xi[:,i]', L_xi[:,i])
        # raise ValueError('stop')
        
        L_xi[N_cis:, :] = L_xi[:N_cis]
        
        print('L_xi shape', L_xi.shape)
        print('L_xi\n', L_xi)
        
        # raise ValueError('stop')
        right_V = L_xi[N_cis:]
        left_V = L_xi[:N_cis]
        
        print('right_V shape', right_V.shape)
        print('right_V\n', right_V)
        # M_plus =  V[:k_dim] @ right_V
        # M_minus = V[:k_dim] @ left_V
        M_plus =  V[:k_dim,:] @ right_V
        M_minus = V[:k_dim,:] @ left_V
        
        print('M_plus shape', M_plus.shape)
        print('M_plus\n', M_plus)
        
        # raise ValueError('stop')
        M = M_minus @ M_plus
        diag = torch.diag(M)
        
        print('M shape', M.shape)
        print('M\n', M)
        
        r_eval, r_evec = torch.linalg.eig(M) # find eigenvalues and eigenvectors
        r_eval = r_eval.real
        r_evec = r_evec.real
        r_eval, r_idx = torch.sort(r_eval, descending=False) # sort eigenvalues in ascending order
        # ! NOT SURE that padding is a good idea
        r_evec = F.pad(r_evec, (0, V.shape[0] - r_evec.shape[1]), 'constant', 0) 
        
        print('r_eval shape', r_eval.shape)
        print('r_eval', r_eval)
        # raise ValueError('stop')
     #   print('reconstructed M from RIGHT\n', r_evec[:, r_idx] @ torch.diag(r_eval) @ r_evec[:, r_idx].inverse())
        
        
        l_eval, l_evec = torch.linalg.eig(M.T) # find left eigenvectors
        l_eval = l_eval.real
        l_evec = l_evec.real
        l_eval, l_idx = torch.sort(l_eval, descending=False) # sort eigenvalues in ascending order
        # ! NOT SURE that padding is a good idea
        l_evec = F.pad(l_evec, (0, V.shape[0] - r_evec.shape[1]), 'constant', 0) 
        
     #   print('reconstructed M from LEFT\n', (l_evec[:, l_idx] @ torch.diag(l_eval) @ l_evec[:, l_idx].inverse()).T)
        
        
        #ic('eigenvalues BEFORE keep_n\n', r_eval.shape)
        e_val_n = r_eval[:keep_n] # keep only the lowest keep_n eigenvalues; full are still stored as e_val
        e_vec_n = r_evec[:, :keep_n]

        resids = torch.zeros(V.shape[0], len(e_val_n)*2) # account for left and right evecs
        #ic(resids.shape)
        # Left = |X-Y>
        # Right = |X+Y>
        # (A+B)|X+Y>
        L_xi_right = torch.zeros((N_rpa, n_V_start), device=device)
        L_xi_left = torch.zeros((N_rpa, n_V_start), device=device)
        for i in range(k_dim):  # i=nd1_old+1,nd1  
             # !should be +/- most likely as A+B, A-B
            L_xi_right[:,i] = form_L_xi(r_evec[:,i], mol, N_cis, N_rpa)
            L_xi_left[:,i] = form_L_xi(l_evec[:,i], mol, N_cis, N_rpa)
            
        L_xi_right[N_cis:, :] = L_xi_right[:N_cis]    #! not sure that needed
        L_xi_left[N_cis:, :] = L_xi_left[:N_cis]
        
        print('L_xi_right\n', L_xi_right[:k_dim, :keep_n])
        print('torch.diag(e_val_n)\n', torch.diag(e_val_n).T)
        print('l_evec[:, :keep_n]', l_evec[:, :keep_n])
        
       # diag = 
        W_l = L_xi_right[:k_dim, :keep_n] - e_val_n * l_evec[:, :keep_n]
        W_r = L_xi_left[N_cis:][:k_dim, :keep_n] - e_val_n * r_evec[:, :keep_n] # check dims
        
        #resids = torch.cat((W_l, W_r), dim=1)
    
        resids_norms_r = torch.tensor([W_r[:,x].norm() for x in range(W_r.shape[1])])
        resids_norms_l = torch.tensor([W_l[:,x].norm() for x in range(W_l.shape[1])])
        # ic(resids_norms, resids_norms.shape)
        if torch.any(resids_norms_r > tol) or torch.any(resids_norms_l > tol): # check if any residual norm is larger than tol
#            print('at least one residual is too big', flush=True)
            mask_r = resids_norms_r >= tol
            mask_l = resids_norms_l >= tol
            large_res_r = W_r[:,mask_r] # residuals larger than tol
            large_res_l = W_l[:,mask_l]
            large_res_r.to(device)
            large_res_l.to(device)
 #           print('large residual norms ', resids_norms[mask])
     #       ic(large_res.shape)
            cor_e_val_r = e_val_n[mask_r] # corresponding eigenvalues !!! check if matches
            cor_e_val_l = e_val_n[mask_l]
           # ic('V shape before adding residuals', V.shape)
            
            if large_res_r.shape[1] <= n_V_max - V.shape[1] or large_res_l.shape[1] <= n_V_max - V.shape[1]: # startin adding residulas only if there is enough space
                
                free_space = n_V_max - V.shape[1]
                i = 0
                j = 0
                while free_space <= n_V_max and i < keep_n and j < keep_n:
                    free_space +=1
                    
                    print('i', i)
                    
                    # right eigenvecs
                    print('diag', diag)
                    print('cor_e_val_r', cor_e_val_r)
                    denom = (diag[i] - cor_e_val_r[i])
                    denom.to(device) 
                    s = large_res_r[:,i]/denom # conditioned residuals > tol
                    s.to(device)
                    s = s/s.norm() # normalize
                    s = F.pad(s, (0, V.shape[1] - s.shape[0]), 'constant', 0) 
                    
                    # !!! PROBABLY HIGHLY INEFFICIENT !!! 
                    # torch QR does not help! Returns square matrix only, we have m, m+1
                    V_new = torch.cat((V, torch.unsqueeze(s, dim=1)), dim=1) # add new vector as column to V
                    V_ort = orthogonalize_torch(V_new) # orthogonalize V space with new vector, again should be translated to PyTorch
                    s_ort = V_ort[:, -1] # orthogonalized against current subspace

                    ratio = s_ort.norm()/resids[:,i].norm() # ratio of old and otogonalized norms
                    if ratio >= tol:
                        V = torch.column_stack((V, s_ort))
                    else:
                        pass
                    i += 1
                    if free_space >= n_V_max: # do not add left if no space
                        break
                    else:
                    # left eigenvecs
                        denom = (diag[j] - cor_e_val_l[j])
                        denom.to(device) 
                        s = large_res_l[:,j]/denom # conditioned residuals > tol
                        s.to(device)
                        s = s/s.norm() # normalize
                        s = F.pad(s, (0, V.shape[1] - s.shape[0]), 'constant', 0) 
                        # !!! PROBABLY HIGHLY INEFFICIENT !!! 
                        # torch QR does not help! Returns square matrix only, we have m, m+1
                        V_new = torch.cat((V, torch.unsqueeze(s, dim=1)), dim=1) # add new vector as column to V
                        V_ort = orthogonalize_torch(V_new) # orthogonalize V space with new vector, again should be translated to PyTorch
                        s_ort = V_ort[:, -1] # orthogonalized against current subspace

                        ratio = s_ort.norm()/resids[:,j].norm() # ratio of old and otogonalized norms
                        if ratio >= tol:
                            V = torch.column_stack((V, s_ort))
                        else:
                            pass                
                        j += 1
            else:
                print('!!!! MAX subspace reached !!!!', flush=True)
              #  ic('current shape of eigenvector matrix', r_evec.shape) 
                # ! TODO: use both LEFT and RIGHT after collapse
                # r_evec_padded = F.pad(r_evec, (0, V.shape[1] - r_evec.shape[1]), 'constant', 0) # pad with zeros to match n_V_max
                # V = V @ r_evec_padded # collapse subspace matrix to n_V_start vectors based on current eigenvectors
                V = r_evec
              #  ic(V.shape, r_evec[:,:n_V_start].shape)
                # ic('forming new subspace from current eigenvectors')
                # ic('subspace after max reached', V.shape)
                continue

        else:
            term = True
            print('============================')
            print('all residuals are below tolerance')
            print('DAVIDSON ALGORITHM CONVERGED')
            print('============================', flush=True)
            # print('eigenvalues: ', e_val, flush=True)
            # print('eigenvectors: ', e_vec, flush=True)
            return r_eval, r_evec

    # runs after big loop if did not converge
    print('============================')
    print('!!! DAVIDSON ALGORITHM DID NOT CONVERGE !!!')
    print('============================', flush=True)
    # print('eigenvalues: ', e_val, flush=True)
    # print('eigenvectors: ', e_vec, flush=True)
    return r_eval, r_evec

In [188]:
eval, _ = davidson(n_V_max = 12, n_iter = 10, tol = 1e-3)

mol Z tensor([6, 1, 1, 1, 1])
rrwork_sorted tensor([17.83759, 17.83759, 17.83759, 18.19470, 18.19470, 18.19470, 18.19470, 18.19470, 18.19470, 18.19470, 18.19470, 18.19470, 34.01030, 34.36740, 34.36740, 34.36740])
indices shape torch.Size([16])
indices tensor([12,  8,  4, 13, 14,  9, 15,  5, 10, 11,  6,  7,  0,  1,  2,  3])
row_idx tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
col_idx tensor([12,  8,  4, 13, 14,  9, 15,  5, 10, 11,  6,  7,  0,  1,  2,  3])
vexp1.shape torch.Size([16, 16])
vexp1 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x6 and 16x6)

In [114]:
torch.sqrt(eval)

tensor([ 5.94858,  6.83013,  9.23851,  9.99968, 11.31665, 12.61708], grad_fn=<SqrtBackward0>)

DAVIDSON without splitting guess density into symmetric and asymmetric parts

!!! SEEMS to be WRONG!!!

In [83]:
davidson_no_split(n_V_max = 12, n_iter = 1, tol = 1e-3)

NameError: name 'davidson_no_split' is not defined

In [82]:
torch.sqrt(eval)

tensor([ 3.48787,  7.85461,  8.22134,  9.26906, 11.49652, 13.05100], grad_fn=<SqrtBackward0>)

## ROUTINES


In [184]:
def form_L_xi_no_split(vexp1, molecule, N_cis, N_rpa, CIS = True):

    
    m = molecule
    gss = m.parameters['g_ss']
    gsp = m.parameters['g_sp']
    gpp = m.parameters['g_pp']
    gp2 = m.parameters['g_p2']
    hsp = m.parameters['h_sp']
    
    mask  = m.mask
    maskd = m.maskd
    idxi  = m.idxi
    idxj  = m.idxj
    nmol  = m.nmol
    molsize = m.molsize
    w       = m.w
    nHeavy = m.nHeavy
    nHydro = m.nHydro
    
    eta = torch.zeros((N_rpa), device=device) 
    
    #print('vexp1.shape', vexp1.shape)
   # print('eta.shape', eta.shape)
    eta[:vexp1.size(0)] = vexp1 # eta is stored as |X|; dcopy?
    
    eta_orig = torch.clone(eta)
    # print('eta_orig.shape', eta_orig.shape)
    # print('eta_orig\n', eta_orig)
    
   # print('m.C MO', m.C_mo[0])
    # print('eta.shape', eta.shape)
    # print(eta)
    eta_ao =  mo2ao(N_cis, eta, m, full=False)     # mo to ao basis (mo2site)
    # print('eta_ao.shape', eta_ao.shape)
    # print(eta_ao)

    # eta_ao_sym, eta_ao_asym = decompose_to_sym_antisym(eta_ao) # decompose to sym and asym

    # Vxi - build 2e integrals in AO basis: G(guess density) in F = H_core + G
    # note density is split into sym and anisym matrices
    # sym is processed as padded 3d array in PYSEQM, see fock module 
    # antisym: 2c-2e works with modified PYSEQM routine; should be antisimmterized afterwards
    # antisym: 1c-2e (diagonal) are taken from NEXMD for now - ugly code with loops
    # TODO: vectorize 1c-2e part
    
    #------------------symmetric------------------------------
    G_sym   =  build_G_sym(eta_ao,
                        gss, gsp, gpp, gp2, hsp,
                        mask, maskd, idxi, idxj, nmol, molsize,
                        w,
                         nHeavy,
                         nHydro)
    # G sym is 1c-2e and 2c-2e of symmetric part of guess density
    
    
    
    # pack 2c-2e part to standard shape
    G_sym = pack.pack(G_sym, nHeavy, nHydro)
    # print('G_sym \n', G_sym)
    
    # G_tot = build_G_antisym(eta_ao, eta_ao_asym, G_sym,
    #                         gss, gsp, gpp, gp2, hsp,
    #                         mask, maskd, idxi, idxj, nmol, molsize,
    #                         w, 
    #                         m,
    #                         nHeavy,
    #                         nHydro)
                        
    # build_G_antisym returns both sym and antisym!
    # TODO: refactor into: 2c-2e antisym, 1c-2e antisym
    # TODO: vectorize 1c-2e antisym, avoid ugly loops
    #! remember about making 2c-2e diagonal 0

    # print('G total \n', G_tot)
    
    # print('============================================')
    # print('Converting Gao full back into MO basis')
    G_mo = ao2mo(N_cis, N_rpa, m, G_sym[0], m.C_mo, full=False) # G in MO basis #! [0] not batched yet
    
    # multiply by MO differencies
    ii=0
    for p in range(m.nocc):
    # print('p', p)
        for h in range(m.nocc, m.norb):
            # print('h', h)
            # print('i', i)
            f = m.e_mo[0][h] - m.e_mo[0][p]
            G_mo[ii] = G_mo[ii] + f * eta_orig[ii]
            G_mo[ii+N_cis] = -G_mo[ii + N_cis] + f * eta_orig[ii+N_cis]
            ii += 1
        
    # print('G_mo.shape', G_mo.shape)
    # print('G_mo\n', G_mo)
    return G_mo

In [183]:
def form_L_xi(vexp1, molecule, N_cis, N_rpa, CIS = True):

    
    m = molecule
    gss = m.parameters['g_ss']
    gsp = m.parameters['g_sp']
    gpp = m.parameters['g_pp']
    gp2 = m.parameters['g_p2']
    hsp = m.parameters['h_sp']
    
    mask  = m.mask
    maskd = m.maskd
    idxi  = m.idxi
    idxj  = m.idxj
    nmol  = m.nmol
    molsize = m.molsize
    w       = m.w
    nHeavy = m.nHeavy
    nHydro = m.nHydro
    
    eta = torch.zeros((N_rpa), device=device) 
    
    #print('vexp1.shape', vexp1.shape)
   # print('eta.shape', eta.shape)
    eta[:vexp1.size(0)] = vexp1 # eta is stored as |X|; dcopy?
    
    eta_orig = torch.clone(eta)
    # print('eta_orig.shape', eta_orig.shape)
    # print('eta_orig\n', eta_orig)
    
   # print('m.C MO', m.C_mo[0])
    # print('eta.shape', eta.shape)
    # print(eta)
    eta_ao =  mo2ao(N_cis, eta, m, full=False)     # mo to ao basis (mo2site)
    # print('eta_ao.shape', eta_ao.shape)
    # print(eta_ao)

    eta_ao_sym, eta_ao_asym = decompose_to_sym_antisym(eta_ao) # decompose to sym and asym

    # Vxi - build 2e integrals in AO basis: G(guess density) in F = H_core + G
    # note density is split into sym and anisym matrices
    # sym is processed as padded 3d array in PYSEQM, see fock module 
    # antisym: 2c-2e works with modified PYSEQM routine; should be antisimmterized afterwards
    # antisym: 1c-2e (diagonal) are taken from NEXMD for now - ugly code with loops
    # TODO: vectorize 1c-2e part
    
    #------------------symmetric------------------------------
    G_sym   =  build_G_sym(eta_ao_sym,
                        gss, gsp, gpp, gp2, hsp,
                        mask, maskd, idxi, idxj, nmol, molsize,
                        w,
                         nHeavy,
                         nHydro)
    
   # print('G_SYM\n', G_sym)
    # G sym is 1c-2e and 2c-2e of symmetric part of guess density
    
    
    
    # pack 2c-2e part to standard shape
    G_sym = pack.pack(G_sym, nHeavy, nHydro)
    # print('G_sym \n', G_sym)
    
    G_tot = build_G_antisym(eta_ao, eta_ao_asym, G_sym,
                            gss, gsp, gpp, gp2, hsp,
                            mask, maskd, idxi, idxj, nmol, molsize,
                            w, 
                            m,
                            nHeavy,
                            nHydro)
    
   # print('G_tot.shape', G_tot.shape)
   # print('G_tot\n', G_tot)                  
    # build_G_antisym returns both sym and antisym!
    # TODO: refactor into: 2c-2e antisym, 1c-2e antisym
    # TODO: vectorize 1c-2e antisym, avoid ugly loops
    #! remember about making 2c-2e diagonal 0

    # print('G total \n', G_tot)
    
    # print('============================================')
    # print('Converting Gao full back into MO basis')
    G_mo = ao2mo(N_cis, N_rpa, m, G_tot[0], m.C_mo, full=False) # G in MO basis #! [0] not batched yet
    
    # print('G_mo.shape', G_mo.shape)
    # print('G_mo\n', G_mo)
    # multiply by MO differencies
    ii=0
    for p in range(m.nocc):
    # print('p', p)
        for h in range(m.nocc, m.norb):
            # print('h', h)
            # print('i', i)
            f = m.e_mo[0][h] - m.e_mo[0][p]
            G_mo[ii] = G_mo[ii] + f * eta_orig[ii]
            G_mo[ii+N_cis] = -G_mo[ii + N_cis] + f * eta_orig[ii+N_cis]
            ii += 1
        
    # print('G_mo.shape', G_mo.shape)
    # print('G_mo\n', G_mo)
    return G_mo

In [9]:
def orthogonalize_torch(U, eps=1e-15):
    """
    Orthogonalizes the matrix U (d x n) using Gram-Schmidt Orthogonalization.
    If the columns of U are linearly dependent with rank(U) = r, the last n-r columns 
    will be 0.
    
    Args:
        U (numpy.array): A d x n matrix with columns that need to be orthogonalized.
        eps (float): Threshold value below which numbers are regarded as 0 (default=1e-15).
    
    Returns:
        (numpy.array): A d x n orthogonal matrix. If the input matrix U's cols were
            not linearly independent, then the last n-r cols are zeros.
    
    Examples:
    ```python
    >>> import numpy as np
    >>> import gram_schmidt as gs
    >>> gs.orthogonalize(np.array([[10., 3.], [7., 8.]]))
    array([[ 0.81923192, -0.57346234],
       [ 0.57346234,  0.81923192]])
    >>> gs.orthogonalize(np.array([[10., 3., 4., 8.], [7., 8., 6., 1.]]))
    array([[ 0.81923192 -0.57346234  0.          0.        ]
       [ 0.57346234  0.81923192  0.          0.        ]])
    ```
    """
    
    n = len(U[0])
    # numpy can readily reference rows using indices, but referencing full rows is a little
    # dirty. So, work with transpose(U)
    V = U.T
    for i in range(n):
        prev_basis = V[0:i]     # orthonormal basis before V[i]
        coeff_vec = prev_basis @ V[i].T  # each entry is np.dot(V[j], V[i]) for all j < i
        # subtract projections of V[i] onto already determined basis V[0:i]
        V[i] -= (coeff_vec @ prev_basis).T
        if torch.norm(V[i]) < eps:
            V[i][V[i] < eps] = 0.   # set the small entries to 0
        else:
            V[i] /= torch.norm(V[i])
    return V.T

In [182]:
def build_G_antisym(eta_ao, eta_ao_asym, G_sym,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w, 
                m,
                nHydro,
                nHeavy):

      # TODO; figure how/why constants are defined in fock_skew
      #!
      #! CHECK
      #!
      # gss = torch.tensor([15.7558, 14.7942, 14.7942])  # GSSII   7.87788  7.39710   7.39710   1/2 # not used
      # gpp = torch.tensor([13.6540,  0.0000,  0.0000])  # GPPII   6.82700   1/2
      # gsp = torch.tensor([10.6212,  0.0000,  0.0000])  # GSPII   10.6211    
      # gp2 = torch.tensor([12.4061,  0.0000,  0.0000])  # GP2II   15.5076
      # hsp = torch.tensor([0.5939, 0.0000, 0.0000])     # HSPII   0.29694   1/2
  
      # see fock_skew in qm_fock in NEXMD
      # create 1d array 
      eta_anti = torch.zeros((m.norb*(m.norb+1)//2), device=device)
      
      # l=0
      # for i in range(m.norb):
      #       # print('i', i)
      #       for j in range(i+1):
      #             print('i j', i, j)

      #             eta_anti[l] = 0.5 * (eta_ao[i,j] - eta_ao[j,i])
      #             l += 1   
      #             # print('l', l) 
      # print('eta_anti NEXMD\n')
      # for e in eta_anti: print(f"{e:.4f}")
      
      eta_anti = torch.zeros((m.norb*(m.norb+1)//2), device=device)
      indices = torch.tril_indices(int(m.norb), int(m.norb), offset = 0)  # Generate the upper triangular indices
      
      # print('indices\n', indices)
      # print('indices[0]\n', indices[0].size())
      eta_anti = 0.5 * (eta_ao[indices[0], indices[1]] - eta_ao[indices[1], indices[0]])

      # print('eta_anti vectorized\n')
      # for e in eta_anti: print(f"{e:.4f}")
      
      
      # print tensor, one element per line
      
      eta_anti_2d = torch.zeros((m.norb, m.norb), device='cpu') 

      #restore to 2d form to build G 2c-2e part
      # l = 0
      # for i in range(0, m.norb): # TODO" vectorize
      #     for j in range(0,i):
      #         l += 1
      #         eta_anti_2d[i,j] += eta_anti[l-1]
      #         eta_anti_2d[j,i] -= eta_anti[l-1]
      #     l += 1 
      # print('eta_anti_2d\n', eta_anti_2d)
      
      # eta_anti_2d_vec = torch.zeros((m.norb, m.norb))     
      
      
      eta_anti_2d[indices[1], indices[0]] = -eta_anti 
      eta_anti_2d = eta_anti_2d - eta_anti_2d.T # antisymmetrize
      
      # print('eta_anti_2d VEC\n', eta_anti_2d)
      #eta_anti_2d_vec.fill_diagonal_(0)

      # print('maskd\n', maskd)

      # TODO: should be vectorized as in build G
      # below explicit working example for H2O taken from NEXMD
      # pascal2 = [1, 3, 6, 10, 15, 21]  # -1 pascal triangle
      # pascal2 = [x -1 for x in pascal2]
      # pascal1 = [0, 1, 3, 6, 10, 15] # -1 for python indexing
      # pascal1 = [x -1 for x in pascal1]
      # orb_loc1 = [0,4,5] # O orbs
      # orb_loc2 = [3,4,5]
      pascal2 = torch.cumsum(torch.arange(1, eta_anti_2d.shape[0]+1), dim=0)  # -1 pascal triangle
      # pascal2 = [-1, 0, 2, 5, 9, 14]
      pascal2 = pascal2 -1 
      # print('pascal2\n', pascal2)
      
      pascal1 = torch.cumsum(torch.arange(0, eta_anti_2d.shape[0]), dim=0) 
      pascal1 = pascal1 - 1
      # pascal1 = [0,  2,  5,  9, 14, 20, 27]
      # pascal1 = [-1, 0, 2, 5, 9, 14]
      # print('pascal1\n', pascal1)
      # print(m.Z)
      
      # ! TODO: write programamtically
      
      # orb_loc1 = [0] # [start of orbitals index of x]
      # orb_loc2 = [1 if m.Z[0] == 1 else 3] # [end of orbitals index of x]
      orb_loc1 = []
      orb_loc2 = []
      # orb_loc1 = [orb_loc1[i - 1] + 4 if m.Z[i] != 1 else orb_loc1[i - 1] + 1 for i, x in enumerate(m.Z)]
      Z = m.Z
      for i,z in enumerate(Z):

        if i == 0:
          orb_loc1.append(0)
          if Z[i] != 1:
            orb_loc2.append(3)
          else:
            orb_loc2.append(0)
          continue
        
        if Z[i-1] != 1:
            orb_loc1.append(orb_loc1[i-1]+4)
        elif Z[i-1] == 1:
            orb_loc1.append(orb_loc1[i-1]+1)
          
        if Z[i] != 1:
            orb_loc2.append(orb_loc2[i-1]+4)
        elif Z[i] == 1:    
            orb_loc2.append(orb_loc2[i-1]+1)
          
             
      # print('orb_loc1\n', orb_loc1)
      # print('orb_loc2\n', orb_loc2)
            
      # orb_loc1 = [0,4,5]
      G_1c2e = torch.zeros((m.norb*(m.norb+1)//2))
    
      for ii in range(molsize): # n_atoms?
      # print('ii', ii)
        if m.Z[ii] == 1:
          pass
        
        else:
          gsp_ii = gsp[ii]
          gpp_ii = gpp[ii]
          gp2_ii = gp2[ii]
          hsp_ii = hsp[ii]
          
          ia = orb_loc1[ii]
          # print('ia', ia)
          ib = orb_loc2[ii]
          
          iplus = ia+1
          ka = pascal2[ia]
          l = ka

          for j in range(iplus, ib+1):
          # print('j', j)
          # print('ia', ia)
          # print('ib', ib)
          # print('l', l)

            mm = l+ia+1
            l = l+j+1
            
            # print(type(mm)) 
            G_1c2e[mm] = G_1c2e[mm] + 0.5*eta_anti[mm] * (hsp_ii - gsp_ii)
            # print('(hsp - gsp)', (hsp - gsp))
            # print('F(M) FIRST', G_1c2e[mm])
           # print('G_1c2e[mm]', G_1c2e[mm])
          #  print('===================')
            
          iminus = ib-1
        
          for j in range(iplus, iminus+1):
            icc = j
            # print('icc', icc)
            for l in range(icc, ib):
              # print('l', l)
              mm = pascal1[l+1] + j+1
              # print('mm', mm)
              # print('(0.25*gpp_ii - 0.6*gp2_ii)', (0.25*gpp_ii - 0.6*gp2_ii) )
              G_1c2e[mm] = G_1c2e[mm]+ eta_anti[mm] * (0.25*gpp_ii - 1.25*0.6*gp2_ii) 
              # print('F(M) SECOND', G_1c2e[mm])
            #  print('G_1c2e[mm]', G_1c2e[mm])
            
      G_1c2e = G_1c2e*2 # antisym 1c2e part
      # print('G_1c2e\n', G_1c2e)
      

      # buils antisymmetric part as Vxi_packA, requires G_sym
      
   #   print('G_sym shape', G_sym.shape)
   #   print('G_sym\n', G_sym)
      
     # print('molsize', molsize)
     
      G_sym = G_sym[0] #! works for one mol only?
      G_sym_orig = G_sym.clone()
      # print('G_sym shape', G_sym.shape)
      # print('G_sym\n', G_sym)
      
      # l = 0
      # for i in range(0, m.norb): # TODO vectorize
      #     for j in range(0,i):
      #         l += 1

      #         G_sym[i,j] += G_1c2e[l-1]
      #         G_sym[j,i] -= G_1c2e[l-1]             
      #     l += 1 # skip diagonal
      # print('G sym after ADDING ANTSYM PART')
      # print('G_sym\n', G_sym)
      
      # pack from 1d to 2d ANTISYM 1c2e part of G
      G_anti_1c2e = torch.zeros(m.norb, m.norb)
      G_anti_1c2e[indices[1], indices[0]] = -G_1c2e
      G_anti_1c2e = G_anti_1c2e - G_anti_1c2e.T
      
      # print('G_anti_1c2e\n', G_anti_1c2e)
      # print(G_anti_1c2e.shape)
      
      # print('SUM')
      # print(G_sym_orig + G_anti_1c2e)
    
      # print('G_anti_1c shape', G_anti_1c.shape)
      # print('G_anti_1c\n', G_anti_1c)
      
    #  print('eta_ao_asym shape', eta_ao_asym.shape)
    #  print('eta_ao_asym\n', eta_ao_asym)
      
      # G_tmp = 2* G_anti_1c + G_sym
      G_sym = torch.unsqueeze(G_sym, 0) #  add dimension

      # build 2c-2e part of antisymmetric G
      # copied from FOCK
      
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      P0 = unpack(eta_ao_asym, nHydro, nHeavy, (nHeavy+nHydro)*4) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      #P = P[...,1,1]+P[...,2,2]+P[...,3,3] #! MODIFIED
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
     # print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      # F0 is still symmetric, probably symmetrized above
      # here we make it antisymmetric back
      #F0 = 2 * F0 
      F0 = pack.pack(F0, m.nHeavy, m.nHydro)
      
      rows, cols = torch.tril_indices(F0.shape[1], F0.shape[2])
      F0[0][rows, cols] *= -1
      F0[0][torch.eye(F0.shape[1]).bool()] *= -1
      
      

      F0[0].diagonal().fill_(0) #! BE WARNED, THIS iS TAKEN FROM OLD NEXMD, PYSEQM produces non-zero diagonal
      F0 = F0*2
    #  print('G ANTISYM shape', F0.shape)
    #  print('G ANTISYM\n', F0*2)
      G_full = G_sym + G_anti_1c2e + F0 # summ of G_sym(sym 1c2e + 2c2e) + antisym 1c2e and 2c2e (F0)
      # print('G_full shape', G_full.shape)
      # print('G_full\n', G_full)
      
      return G_full

In [11]:
def build_G_sym(M_ao,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w,
                nHydro,
                nHeavy):
    
    
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      # # TODO: feed params programmatically
      
      P0 = unpack(M_ao, nHydro, nHeavy, (nHeavy+nHydro)*4) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      # print('P0.shape', P0.shape)
      # print('P0\n', P0)
      #---------------fill diagonal 1c-2e -------------------
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      # print('P.shape', P.shape)
      # print('P\n', P)
      
      Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      ## http://openmopac.net/manual/1c2e.html
    #  (s,s)
      TMP = torch.zeros_like(F)
      TMP[maskd,0,0] = 0.5*P[maskd,0,0]*gss + Pptot[maskd]*(gsp-0.5*hsp)
      for i in range(1,4):
          #(p,p)
          TMP[maskd,i,i] = P[maskd,0,0]*(gsp-0.5*hsp) + 0.5*P[maskd,i,i]*gpp \
                          + (Pptot[maskd] - P[maskd,i,i]) * (1.25*gp2-0.25*gpp)
          #(s,p) = (p,s) upper triangle
          TMP[maskd,0,i] = P[maskd,0,i]*(1.5*hsp - 0.5*gsp)
      #(p,p*)
      for i,j in [(1,2),(1,3),(2,3)]:
          TMP[maskd,i,j] = P[maskd,i,j]* (0.75*gpp - 1.25*gp2)

      F.add_(TMP)
      
           
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
      #print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      F0 = 2 * F0 #! BE CAREFUL
      # print('F0.shape', F0.shape)
      # print(F0) 
      
      return F0

In [12]:
def decompose_to_sym_antisym(A):
        

    A_sym = 0.5 * (A + A.T)
    A_antisym = 0.5 * (A - A.T)
    
    return A_sym, A_antisym

In [57]:
def ao2mo(N_cis, N_rpa, molecule, M_ao, C, full=False):
    """
    transform matrix from AO to MO basis

    Parameters
    ----------
    M_AO : torch tensor # TODO add size
        matrix in AO basis
    C : torch tensor # TODO add size
        matrix of MO coefficients # TODO row or columns, structure?
        
    Returns
    -------
    M_MO : torch tensor # TODO add size
        matrix in MO basis
    """    
    m = molecule
    
    if full == True:
        M_mo = C.T @ M_ao @ C
        return M_mo
        
    else:
         # COPY of subroutine site2mo from Lioville
         
        G_ao = M_ao # TODO rename
        
        # eta1 = eta1.view(-1, m.nvirt[0]) # 1d -> 2d
        # print(eta1.shape)
        # print('eta1', eta1)
        # print('==============')
        
        eta_mo = torch.zeros((N_rpa))
       # eta_mo = torch.zeros((m.norb, m.norb), device=device)

        dgemm1 = G_ao.T @ m.C_mo[0]

        # print('dgemm1.shape', dgemm1.shape)
        # print(dgemm1)
        
        dgemm2 =  m.C_mo[0][:, m.nocc:m.norb].T @ dgemm1[:,:m.nocc]
        

        dgemm2 = dgemm2.T.flatten()
        eta_mo[:dgemm2.size(0)] = dgemm2 
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        dgemm3 =  dgemm1[:, m.nocc:].T @ m.C_mo[0][:, :m.nocc]
        
        # print('dgemm3.T.shape', dgemm3.T.shape)
        # print(dgemm3.T)
        
        eta_mo[N_cis:] = dgemm3.T.flatten() 
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)

        M_mo = eta_mo
    
    return M_mo

In [58]:
def mo2ao(N_cis, M_mo, molecule, full=False):
    """
    transform matrix from AO to MO basis

    Parameters
    ----------
    M_AO : torch tensor # TODO add size
        matrix in AO basis
    C : torch tensor # TODO add size
        matrix of MO coefficients # TODO row or columns, structure?
        
    Returns
    -------
    M_MO : torch tensor # TODO add size
        matrix in MO basis
    """    
    m = molecule
    
    # print('m C_mo', m.C_mo)
    if full == True:
        M_ao = C.T @ M_mo @ C #! does not currently work
        
        return M_ao
    else:
        
        eta = M_mo # TODO rename
        
        eta1 = eta[:N_cis]
        eta1 = eta1.view(-1, m.nvirt[0]) # 1d -> 2d
        # print(eta1.shape)
        # print('eta1', eta1)
        # print('==============')
        
        eta_mo = torch.zeros((m.norb, m.norb), device=device)

        dgemm1 = eta1 @ m.C_mo[0][:, m.nocc:m.norb].T # operations on |X| ?

        # print('dgemm1.shape', dgemm1.shape)
        # print(dgemm1)
        
        eta_mo[:m.nocc] = dgemm1
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        
        eta2 = eta[N_cis:]                            # operations on |Y| ?
        eta2 = eta2.view(-1, m.nvirt[0]) # 1d -> 2d
    
        dgemm2 = eta2.T @ m.C_mo[0][:, :m.nocc].T
        
        # print('dgemm2.shape', dgemm2.shape)
        # print(dgemm2)
        
        eta_mo[m.nocc:] = dgemm2
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        dgemm3 = m.C_mo[0] @ eta_mo
        eta_ao = dgemm3 
    
    
        return eta_ao